<a href="https://colab.research.google.com/github/Filin546/-/blob/master/%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install SpeechRecognition

In [35]:
!pip install pydub

In [36]:
!pip install python-docx

In [37]:
!pip install pyspellchecker

In [38]:
!pip install vaderSentiment

In [39]:
!pip install pyLDAvis

In [40]:
import os
import shutil
from docx import Document
from pydub import AudioSegment
from speech_recognition import UnknownValueError
import speech_recognition as sr
from spellchecker import SpellChecker
from pydub.exceptions import CouldntDecodeError
from speech_recognition import AudioFile, Recognizer, UnknownValueError

Функция для конвертации файлов в формат WAV

In [41]:
def convert_to_wav(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    total_files = len([filename for filename in os.listdir(input_folder) if filename.endswith(".mp3")])
    processed_files =0

    for filename in os.listdir(input_folder):
        if filename.endswith(".mp3"):
            input_file = os.path.join(input_folder, filename)
            output_folder_wav = os.path.join(output_folder, os.path.splitext(filename)[0])
            if not os.path.exists(output_folder_wav):
                os.makedirs(output_folder_wav)
            output_file = os.path.join(output_folder_wav, os.path.splitext(filename)[0] + ".wav")
            if not os.path.exists(output_file):
                audio = AudioSegment.from_mp3(input_file)
                audio.export(output_file, format="wav")
            split_audio(output_file, output_folder)
            processed_files +=1
            print(f"Converted {processed_files}/{total_files} files to WAV format.")

Функция для разделения аудиофайла на части

In [42]:
def split_audio(audio_file, output_folder):
    audio = AudioSegment.from_wav(audio_file)
    segment_length_ms =10000
    for i, start_time in enumerate(range(0, len(audio), segment_length_ms)):
        segment = audio[start_time:start_time + segment_length_ms]
        file_name = os.path.splitext(os.path.basename(audio_file))[0] + f"_{i+1}.wav"
        folder_name = os.path.splitext(os.path.basename(audio_file))[0]
        folder_path = os.path.join(output_folder, folder_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        output_file = os.path.join(folder_path, file_name)
        if not os.path.exists(output_file):
            segment.export(output_file, format="wav")
            print(f"Split {audio_file} into {i+1} parts.")

            # Transcribe the audio segment
            try:
                transcription = transcribe_audio_segment(output_file)
                if transcription:
                    save_transcription_to_doc(transcription, folder_name, output_folder_transcription)
                else:
                    print(f"Failed to transcribe {output_file}")
            except UnknownValueError:
                print(f"UnknownValueError: Failed to transcribe {output_file}")
        else:
            print(f"Skipping {output_file} as it's already transcribed.")

Функция для транскрибации аудиофайла

In [43]:
def transcribe_audio_segment(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source, duration=None)  # Read the entire audio file
        text = recognizer.recognize_google(audio_data, language="ru-RU", show_all=False)
        return text

Функция для сохранения транскрибации в документ

In [44]:
def save_transcription_to_doc(transcription, folder_name, output_folder_transcription):
    output_folder_transcription = os.path.join(output_folder_transcription, folder_name)
    if not os.path.exists(output_folder_transcription):
        os.makedirs(output_folder_transcription)

    doc = Document()

    # Исправляем орфографию
    spell = SpellChecker(language='ru')
    corrected_transcription = ' '.join([spell.correction(word) if spell.correction(word) is not None else word for word in transcription.split()])

    doc.add_paragraph(corrected_transcription)
    file_count = len([f for f in os.listdir(output_folder_transcription) if f.endswith('.doc')])
    doc_path = os.path.join(output_folder_transcription, f"{folder_name}_{file_count + 1}.doc")
    doc.save(doc_path)  # Corrected save method
    print(f"Transcription saved to {doc_path}")

In [45]:
def combine_transcriptions(input_folder, output_folder):
    # Создание папки для объединенных документов, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Итерация по всем подпапкам в папке с результатами транскрипции
    for folder_name in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder_name)
        if os.path.isdir(folder_path):
            # Проверка, существует ли уже объединенный документ для текущей подпапки
            combined_doc_name = f"общий_документ_{folder_name}.doc"
            combined_doc_path = os.path.join(output_folder, combined_doc_name)
            if os.path.exists(combined_doc_path):
                print(f"Общий документ для подпапки {folder_name} уже существует.")
                continue

            # Создание нового документа
            doc = Document()

            # Чтение всех файлов в подпапке и добавление их содержимого в новый документ
            for filename in os.listdir(folder_path):
                if filename.endswith('.doc'): # Предполагается, что файлы с текстом имеют расширение .doc
                    file_path = os.path.join(folder_path, filename)
                    # Чтение содержимого документа
                    doc_content = Document(file_path)
                    for paragraph in doc_content.paragraphs:
                        doc.add_paragraph(paragraph.text)

            # Сохранение объединенного документа
            doc.save(combined_doc_path)
            print(f"Общий документ для подпапки {folder_name} сохранен в {combined_doc_path}")

In [46]:
def categorize_dialogues(input_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define categories
    categories = {
        "конфликтные": ["конфликт", "недовольство", "проблема"],
        "информационные": ["информация", "вопрос", "подробности"],
        "с вопросами": ["вопрос", "спрашивать", "уточнение"]
    }

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".doc"):
            input_file_path = os.path.join(input_folder, filename)
            doc = Document(input_file_path)
            text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])

            # Categorize dialogue based on keywords
            categorized = False
            for category, keywords in categories.items():
                for keyword in keywords:
                    if keyword in text.lower():
                        category_folder = os.path.join(output_folder, category)
                        if not os.path.exists(category_folder):
                            os.makedirs(category_folder)
                        _output_file_path = os.path.join(category_folder, filename)
                        # Check if file already exists in the destination folder
                        if not os.path.exists(_output_file_path):
                            # Copy the file to the corresponding category folder
                            shutil.copy(input_file_path, _output_file_path)
                        categorized = True
                        break
                if categorized:
                    break

            # If not categorized into any predefined category, move to "other" folder
            if not categorized:
                other_folder = os.path.join(output_folder, "other")
                if not os.path.exists(other_folder):
                    os.makedirs(other_folder)
                _output_file_path = os.path.join(other_folder, filename)
                # Check if file already exists in the destination folder
                if not os.path.exists(_output_file_path):
                    shutil.copy(input_file_path, _output_file_path)
                    print(f"Dialog {filename} moved to 'other' category.")

Основная часть: конвертация, разделение и транскрибация аудиофайлов

In [47]:
# Путь к папке с исходными файлами
input_folder = "/content/drive/MyDrive/dataset_proect/звонки"
output_folder = "/content/drive/MyDrive/dataset_proect/формат_wav"
output_folder_transcription = "/content/drive/MyDrive/dataset_proect/транскрибации"
input_folder_transcription = "/content/drive/MyDrive/dataset_proect/транскрибации"
output_folder_combined = "/content/drive/MyDrive/dataset_proect/общие_документы"
input_folder_combined = "/content/drive/MyDrive/dataset_proect/общие_документы"
output_folder_categorized = "/content/drive/MyDrive/dataset_proect/категоризированные_диалоги"

# Вызов функций
convert_to_wav(input_folder, output_folder)
combine_transcriptions(input_folder_transcription, output_folder_combined)
categorize_dialogues(input_folder_combined, output_folder_categorized)

Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_1.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_2.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_3.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_4.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_5.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_6.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_7.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_8.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_9.wav as it's already transcribed.
Skipping /content/drive/MyDrive/dataset_proect/формат_wav/01/01_10.wav as it's already transcribed.
Skipping 